<a href="https://colab.research.google.com/github/ehdwls314/prac_class/blob/main/%EB%8D%B0%EB%A7%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb


In [ ]:
# 모델 학습

# X 는 train에서 quality 를 제외한 모든 변수
# y 는 train의 qulity 변수

X = train_one.drop('quality',axis= 1)
y = train_one['quality']

# fit 메소드를 이용해 모델 학습
model.fit(X,y)

# predict 메소드와 test_one 데이터를 이용해 품질 예측
pred = model.predict(test_one)

# sample_submission.csv 파일을 불러와 예측된 값으로 채워 주기
submission = pd.read_csv('data/sample_submission.csv')
submission['quality'] = pred
submission.head()

# 정답파일 내보내기
submission.to_csv('lgbm_pred.csv',index = False)

In [2]:
data = pd.read_csv('한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')

In [3]:
data.head()

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105


In [4]:
data['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [5]:
d_map = {}
for i, d in enumerate(data['구분'].unique()):
    d_map[d] = i
data['구분'] = data['구분'].map(d_map)

In [6]:
data['연월일'] = pd.to_datetime(data['연월일'])

In [7]:
data['year'] = data['연월일'].dt.year
data['month'] = data['연월일'].dt.month
data['day'] = data['연월일'].dt.day
data['weekday'] = data['연월일'].dt.weekday

In [8]:
data_A = data[data['구분']==0]
data_B = data[data['구분']==1]
data_C = data[data['구분']==2]
data_D = data[data['구분']==3]
data_E = data[data['구분']==4]
data_G = data[data['구분']==5]
data_H = data[data['구분']==6]

In [9]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

In [10]:
train = data[data['year'].isin(train_years)]
val = data[data['year'].isin(val_years)]

In [11]:
features = ['구분', 'month', 'day', 'weekday', '시간']
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']


In [16]:
train_data = lgb.Dataset(train_x, label = train_y)
val_data = lgb.Dataset(val_x, label = val_y)

In [17]:
params = {
    'learning_rate':0.01,
    'max_depth': 16,
    'boosting': 'gbdt', 
    'task' : 'train',
    'objective': 'regression',
    'metric':'mae',
    'seed':42
}

model = lgb.train(params, train_data, 500, val_data, verbose_eval=20, early_stopping_rounds=10)

Training until validation scores don't improve for 10 rounds.
[20]	valid_0's l1: 640.928
[40]	valid_0's l1: 548.056
[60]	valid_0's l1: 473.709
[80]	valid_0's l1: 414.424
[100]	valid_0's l1: 367.071
[120]	valid_0's l1: 330.458
[140]	valid_0's l1: 302.294
[160]	valid_0's l1: 279.668
[180]	valid_0's l1: 262.608
[200]	valid_0's l1: 249.342
[220]	valid_0's l1: 236.454
[240]	valid_0's l1: 225.916
[260]	valid_0's l1: 216.131
[280]	valid_0's l1: 207.904
[300]	valid_0's l1: 200.724
[320]	valid_0's l1: 194.721
[340]	valid_0's l1: 189.414
[360]	valid_0's l1: 184.653
[380]	valid_0's l1: 180.603
[400]	valid_0's l1: 177.055
[420]	valid_0's l1: 174.083
[440]	valid_0's l1: 171.437
[460]	valid_0's l1: 169.039
[480]	valid_0's l1: 167.047
[500]	valid_0's l1: 165.283
Did not meet early stopping. Best iteration is:
[500]	valid_0's l1: 165.283


In [15]:
predict_train = model.predict(train_x)

In [21]:
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [23]:
test.head()

,일자|시간|구분
0,2019-01-01 01 A
1,2019-01-01 02 A
2,2019-01-01 03 A
3,2019-01-01 04 A
4,2019-01-01 05 A


In [24]:
submission.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,0
1,2019-01-01 02 A,0
2,2019-01-01 03 A,0
3,2019-01-01 04 A,0
4,2019-01-01 05 A,0


In [26]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [27]:
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday

In [28]:
test['구분'] = test['구분'].map(d_map)

In [29]:
test_x = test[features]

In [30]:
test_x

,구분,month,day,weekday,시간
0,0,1,1,1,1
1,0,1,1,1,2
2,0,1,1,1,3
3,0,1,1,1,4
4,0,1,1,1,5
...,...,...,...,...,...
15115,6,3,31,6,20
15116,6,3,31,6,21
15117,6,3,31,6,22
15118,6,3,31,6,23


In [31]:
preds = model.predict(test_x)

In [32]:
submission['공급량'] = preds

In [36]:
submission.to_csv('/content/drive/MyDrive/데마/submission.csv', index=False)


In [ ]:
# sample_submission.csv 파일을 불러와 예측된 값으로 채워 주기

submission = pd.read_csv('data/sample_submission.csv')

submission['quality'] = pred

submission.head()

In [ ]:
pred = model.predict(train_x, train_y)

AttributeError: ignored

In [ ]:
history_size = 30*24*3
target_size = 2160

y_A = y_A.loc[y_A.date >= datetime(2016, 1, 1)]
weather = weather[-len(y_A):]

y_A = pd.concat([y_A[['supply']], weather[['체감온도']]], axis=1)

X_train, y_train = prepare_data(y_A, history_size, target_size)


In [ ]:
history_size = 30*24*3
target_size = 2160


In [ ]:
x_test = []
x_test.append(total_A.values[-history_size:])
x_test = np.array(x_test)
x_test.shape

(1, 2160, 8)

In [ ]:
model.fit(train_x, train_y)

AttributeError: ignored

In [ ]:
best_lgbm = get_best_params(model, params)

NameError: ignored

In [ ]:
train.groupby(['month'])['공급량'].mean()

month
1     1791.931346
2     1622.932402
3     1210.662557
4      799.846869
5      542.799101
6      474.749196
7      471.275057
8      444.279738
9      460.747752
10     611.115468
11    1137.585059
12    1684.639508
Name: 공급량, dtype: float64

In [ ]:
train.groupby(['구분'])['공급량'].mean()

구분
0    1157.195134
1     949.208549
2     132.983272
3     656.975075
4    1372.319323
5    2028.477825
6     246.889076
Name: 공급량, dtype: float64